In [1]:
!pip install pydot
!pip install graphviz

In [2]:

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report


In [3]:
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
train_path = 'seg_train/seg_train/'
test_path = 'seg_test/seg_test/'

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [6]:
val_test_datagen = ImageDataGenerator(rescale=1./255)


In [7]:
train_data = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 14034 images belonging to 6 classes.


In [8]:
val_data = val_test_datagen.flow_from_directory(
    test_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 3000 images belonging to 6 classes.


In [9]:
test_data = val_test_datagen.flow_from_directory(
    test_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 3000 images belonging to 6 classes.


In [10]:
# Build the CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

In [11]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
# Train the model
history = model.fit(train_data, epochs=10, validation_data=val_data)


Epoch 1/10
439/439 [==============================] - 205s 465ms/step - loss: 1.0434 - accuracy: 0.6087 - val_loss: 0.7554 - val_accuracy: 0.7233
Epoch 2/10
439/439 [==============================] - 190s 433ms/step - loss: 0.7352 - accuracy: 0.7287 - val_loss: 0.6678 - val_accuracy: 0.7633
Epoch 3/10
439/439 [==============================] - 190s 432ms/step - loss: 0.6376 - accuracy: 0.7670 - val_loss: 0.6156 - val_accuracy: 0.7923
Epoch 4/10
439/439 [==============================] - 191s 434ms/step - loss: 0.5622 - accuracy: 0.7999 - val_loss: 0.6427 - val_accuracy: 0.7883
Epoch 5/10
439/439 [==============================] - 194s 441ms/step - loss: 0.5089 - accuracy: 0.8134 - val_loss: 0.6225 - val_accuracy: 0.7890
Epoch 6/10
439/439 [==============================] - 187s 427ms/step - loss: 0.4720 - accuracy: 0.8331 - val_loss: 0.4855 - val_accuracy: 0.8430
Epoch 7/10
439/439 [==============================] - 177s 402ms/step - loss: 0.4460 - accuracy: 0.8397 - val_loss: 0.5253 -

In [13]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_data)
print(f'Test Accuracy: {test_acc}')

# Confusion Matrix
predictions = model.predict(test_data)
conf_matrix = confusion_matrix(test_data.classes, np.argmax(predictions, axis=1))

# Display confusion matrix
print('Confusion Matrix:')
print(conf_matrix)

# Classification Report
class_report = classification_report(test_data.classes, np.argmax(predictions, axis=1))
print('Classification Report:')
print(class_report)


94/94 [==============================] - 6s 68ms/step - loss: 0.5020 - accuracy: 0.8360
Test Accuracy: 0.8360000252723694
94/94 [==============================] - 7s 67ms/step
Confusion Matrix:
[[323   9   5  19  20  61]
 [  1 458   4   9   1   1]
 [  4   4 422  88  32   3]
 [  1   2  50 429  43   0]
 [  8   3  27  20 445   7]
 [ 35  18   6   7   4 431]]
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.74      0.80       437
           1       0.93      0.97      0.95       474
           2       0.82      0.76      0.79       553
           3       0.75      0.82      0.78       525
           4       0.82      0.87      0.84       510
           5       0.86      0.86      0.86       501

    accuracy                           0.84      3000
   macro avg       0.84      0.84      0.84      3000
weighted avg       0.84      0.84      0.84      3000



In [14]:
# Display model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 82944)             0         
                                                                 
 dense (Dense)               (None, 128)               1

In [24]:
from sklearn.model_selection import StratifiedShuffleSplit

# Specify the number of folds
n_folds = 10

# Get the data and labels
X = train_data[0][0]
y = train_data[0][1]

# Convert labels to integer indices
y_indices = np.argmax(y, axis=1)

# Initialize StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=n_folds, test_size=len(np.unique(y_indices)), random_state=42)


# Iterate through folds
for fold, (train_index, val_index) in enumerate(sss.split(X, y_indices), 1):
    print(f"\nFold {fold}/{n_folds}")

    # Create the CNN model
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(6, activation='softmax')
    ])
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Split the data into train and validation sets
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Create data generators for train and validation sets
    train_data_fold = train_datagen.flow(X_train, y_train, batch_size=32)
    val_data_fold = train_datagen.flow(X_val, y_val, batch_size=32)

    # Train the model
    history = model.fit(train_data_fold, epochs=10, validation_data=val_data_fold)

    # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(test_data)
    print(f'Test Accuracy for Fold {fold}: {test_acc}')

    # Display model summary
    model.summary()



Fold 1/10
Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 1.7918 - accuracy: 0.1538 - val_loss: 1.7787 - val_accuracy: 0.3333
Epoch 2/10
1/1 [==============================] - 1s 641ms/step - loss: 1.7831 - accuracy: 0.2692 - val_loss: 1.7185 - val_accuracy: 0.3333
Epoch 3/10
1/1 [==============================] - 1s 599ms/step - loss: 1.7530 - accuracy: 0.2692 - val_loss: 1.6832 - val_accuracy: 0.3333
Epoch 4/10
1/1 [==============================] - 1s 579ms/step - loss: 1.7670 - accuracy: 0.2692 - val_loss: 1.6915 - val_accuracy: 0.3333
Epoch 5/10
1/1 [==============================] - 1s 561ms/step - loss: 1.7457 - accuracy: 0.2692 - val_loss: 1.7120 - val_accuracy: 0.3333
Epoch 6/10
1/1 [==============================] - 1s 558ms/step - loss: 1.7447 - accuracy: 0.2692 - val_loss: 1.7195 - val_accuracy: 0.3333
Epoch 7/10
1/1 [==============================] - 1s 550ms/step - loss: 1.7452 - accuracy: 0.2692 - val_loss: 1.7148 - val_accuracy: 0.3333
Epoch 8/10
1

In [29]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

def simplified_alexnet(input_shape, num_classes):
    model = models.Sequential()

    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    optimizer = Adam(learning_rate=0.0001) 

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model


input_shape = (64, 64, 3)  
num_classes = 6  

model = simplified_alexnet(input_shape, num_classes)
model.summary()


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_31 (Conv2D)          (None, 62, 62, 64)        1792      
                                                                 
 max_pooling2d_27 (MaxPooli  (None, 31, 31, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_32 (Conv2D)          (None, 29, 29, 128)       73856     
                                                                 
 max_pooling2d_28 (MaxPooli  (None, 14, 14, 128)       0         
 ng2D)                                                           
                                                                 
 conv2d_33 (Conv2D)          (None, 12, 12, 256)       295168    
                                                                 
 conv2d_34 (Conv2D)          (None, 10, 10, 256)     

In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data generators
train_data = train_datagen.flow_from_directory(
    train_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

val_data = val_test_datagen.flow_from_directory(
    test_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

test_data = val_test_datagen.flow_from_directory(
    test_path,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


input_shape = (64, 64, 3)  
num_classes = 6 

model = simplified_alexnet(input_shape, num_classes)
model.summary()

# Train the model
history = model.fit(train_data, epochs=10, validation_data=val_data)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_data)
print(f'Test Accuracy: {test_acc}')

# Display model summary
model.summary()


Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_35 (Conv2D)          (None, 62, 62, 64)        1792      
                                                                 
 max_pooling2d_30 (MaxPooli  (None, 31, 31, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_36 (Conv2D)          (None, 29, 29, 128)       73856     
                                                                 
 max_pooling2d_31 (MaxPooli  (None, 14, 14, 128)       0         
 ng2D)                                                           
                                                                 
 conv2d_37 (Conv2D)          (None, 12, 12, 256)       295